# Fedbiomed Researcher Listing Datasets and Selecting Particular Nodes

Use for developing (autoreloads changes made across packages)

In [1]:
%load_ext autoreload
%autoreload 2

In this tutorial, you will learn how to list datasets deployed in nodes and select them to perform an experiement. To be able to follow this example, you need to lauch more than 2 nodes that have MNIST dataset.

## Setting the client up
It is necessary to previously configure multiple node:
1. `./scripts/fedbiomed_run node config config-n1.ini add`
  * Select option 2 (default) to add MNIST to the client
  * Confirm default tags by hitting "y" and ENTER
  * Pick the folder where MNIST is downloaded (this is due torch issue https://github.com/pytorch/vision/issues/3549)
  * Data must have been added (if you get a warning saying that data must be unique is because it's been already added)
  * Start node with `./scripts/fedbiomed_run node config config-n1.ini start`  
  
  
2. Add data to seconda node: 
    * Open new terminal create new node by indicating the MNIST dataset that you already dowloaded
    `./scripts/fedbiomed_run node config config-n2.ini --add-mnist path/to/your/mnist/data`
    * Start node: `./scripts/fedbiomed_run node config config-n2.ini start`
3. Add a third node by following the same instructions of step 2.  

## List Datasets Available in Nodes

You can easly list dataset located in online nodes using `list()` method of `Request` class. 

**Arguments**  

 * `verbose` : Prints list of datasets in table format 
 * `client`  : Array includes client ids. Gets list of dataset only given client ids  
 
 

In [2]:
from fedbiomed.researcher.requests import Requests

req = Requests()
datasets = req.list(verbose=True)


You can also access these information from result of the `list()` method. 

In [3]:
print('Datasets -----------------------------  ')
print(datasets)
print('Node ids -----------------------------  ')
print(datasets.keys())


Datasets -----------------------------  
{'node_2034e2c7-9db4-4adc-93d5-21f07046c680': [{'name': 'MNIST', 'data_type': 'default', 'tags': ['#MNIST', '#dataset'], 'description': 'MNIST database', 'shape': [60000, 1, 28, 28]}], 'node_5c5a2948-b12c-4102-9925-49c03a2d5348': [{'name': 'MNIST', 'data_type': 'default', 'tags': ['#MNIST', '#dataset'], 'description': 'MNIST database', 'shape': [60000, 1, 28, 28]}]}
Node ids -----------------------------  
dict_keys(['node_2034e2c7-9db4-4adc-93d5-21f07046c680', 'node_5c5a2948-b12c-4102-9925-49c03a2d5348'])


In [8]:
req = Requests()
datasets = req.list(clients=['node_2034e2c7-9db4-4adc-93d5-21f07046c680'], verbose=True)

You can create a list that contains nodes ids that you want run your experiment. After that you need to initialize your Experiment with the node (client) id list.  

**IMPORTANT:** Please change values based on your listing result

In [9]:
clients = ['node_2034e2c7-9db4-4adc-93d5-21f07046c680']

After specifying nodes in the `clients` list. You can start creating your model and experiment.

## Create a Model and an Experiment

Declare a torch.nn MyTrainingPlan class to send for training on the node

In [10]:
import os
from fedbiomed.researcher.environ import environ
import tempfile
tmp_dir_model = tempfile.TemporaryDirectory(dir=environ['TMP_DIR'])
model_file = os.path.join(tmp_dir_model.name, 'class_export_mnist.py')

Note : write **only** the code to export in the following cell

In [11]:
%%writefile "$model_file"

import torch
import torch.nn as nn
from fedbiomed.common.torchnn import TorchTrainingPlan
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Here we define the model to be used. 
# You can use any class name (here 'Net')
class MyTrainingPlan(TorchTrainingPlan):
    def __init__(self):
        super(MyTrainingPlan, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        # In this case, we need the torch DataLoader classes
        # Since we will train on MNIST, we need datasets and transform from torchvision
        deps = ["from torchvision import datasets, transforms",
               "from torch.utils.data import DataLoader"]
        self.add_dependency(deps)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

    def training_data(self, batch_size = 48):
        # Custom torch Dataloader for MNIST data
        transform = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])
        dataset1 = datasets.MNIST(self.dataset_path, train=True, download=False, transform=transform)
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        data_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
        return data_loader
    
    def training_step(self, data, target):
        output = self.forward(data)
        loss   = torch.nn.functional.nll_loss(output, target)
        return loss


Writing /Users/jls/Development/fedbiomed/fedbiomed/var/tmp/tmp6cman9n8/class_export_mnist.py


This group of arguments correspond respectively:
* `model_args`: a dictionary with the arguments related to the model (e.g. number of layers, features, etc.). This will be passed to the model class on the client side.
* `training_args`: a dictionary containing the arguments for the training routine (e.g. batch size, learning rate, epochs, etc.). This will be passed to the routine on the client side.

**NOTE:** typos and/or lack of positional (required) arguments will raise error. 🤓

In [12]:
model_args = {}

training_args = {
    'batch_size': 48, 
    'lr': 1e-3, 
    'epochs': 1, 
    'dry_run': False,  
    'batch_maxnum': 100 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

Define an experiment
- search nodes serving data for these `tags`, optionally filter on a list of client ID with `clients`
- run a round of local training on nodes with model defined in `model_path` + federation with `aggregator`
- run for `rounds` rounds, applying the `client_selection_strategy` between the rounds

In [13]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['#MNIST', '#dataset']
rounds = 2

exp = Experiment(tags=tags,
                 clients=clients,
                 model_path=model_file,
                 model_args=model_args,
                 model_class='MyTrainingPlan',
                 training_args=training_args,
                 rounds=rounds,
                 aggregator=FedAverage(),
                 client_selection_strategy=None)

Let's start the experiment.

By default, this function doesn't stop until all the `rounds` are done for all the clients

In [14]:
exp.run()

Local training results for each round and each node are available in `exp.training_replies` (index 0 to (`rounds` - 1) ).

For example you can view the training results for the last round below.

Different timings (in seconds) are reported for each dataset of a node participating in a round :
- `rtime_training` real time (clock time) spent in the training function on the node
- `ptime_training` process time (user and system CPU) spent in the training function on the node
- `rtime_total` real time (clock time) spent in the researcher between sending the request and handling the response, at the `Job()` layer

In [15]:
print("\nList the training rounds : ", exp.training_replies.keys())

print("\nList the clients for the last training round and their timings : ")
round_data = exp.training_replies[rounds - 1].data
for c in range(len(round_data)):
    print("\t- {id} :\
    \n\t\trtime_training={rtraining:.2f} seconds\
    \n\t\tptime_training={ptraining:.2f} seconds\
    \n\t\trtime_total={rtotal:.2f} seconds".format(id = round_data[c]['node_id'],
        rtraining = round_data[c]['timing']['rtime_training'],
        ptraining = round_data[c]['timing']['ptime_training'],
        rtotal = round_data[c]['timing']['rtime_total']))
print('\n')
    
exp.training_replies[rounds - 1].dataframe


List the training rounds :  dict_keys([0, 1])

List the clients for the last training round and their timings : 
	- node_2034e2c7-9db4-4adc-93d5-21f07046c680 :    
		rtime_training=10.05 seconds    
		ptime_training=11.76 seconds    
		rtime_total=20.01 seconds




,success,msg,dataset_id,node_id,params_path,params,timing
0,True,,dataset_a5e182c2-2a63-4aee-aa51-7468e4c51508,node_2034e2c7-9db4-4adc-93d5-21f07046c680,/Users/jls/Development/fedbiomed/fedbiomed/var...,"{'conv1.weight': [[tensor([[-0.1146, -0.3210, ...","{'rtime_training': 10.052962767000054, 'ptime_..."


Federated parameters for each round are available in `exp.aggregated_params` (index 0 to (`rounds` - 1) ).

For example you can view the federated parameters for the last round of the experiment :

In [16]:
print("\nList the training rounds : ", exp.aggregated_params.keys())

print("\nAccess the federated params for the last training round :")
print("\t- params_path: ", exp.aggregated_params[rounds - 1]['params_path'])
print("\t- parameter data: ", exp.aggregated_params[rounds - 1]['params'].keys())



List the training rounds :  dict_keys([0, 1])

Access the federated params for the last training round :
	- params_path:  /Users/jls/Development/fedbiomed/fedbiomed/var/tmp/researcher_params_1c07414c-0721-4ece-95c3-4be256e906ea.pt
	- parameter data:  odict_keys(['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias'])


## Optional : searching the data

In [17]:
from fedbiomed.researcher.requests import Requests

r = Requests()
data = r.search(tags)

import pandas as pd
for node_id in data.keys():
    print('\n','Data for ', node_id, '\n\n', pd.DataFrame(data[node_id]))


 Data for  node_5c5a2948-b12c-4102-9925-49c03a2d5348 

     name data_type                tags     description               shape  \
0  MNIST   default  [#MNIST, #dataset]  MNIST database  [60000, 1, 28, 28]   

                                     dataset_id dtypes  
0  dataset_319d0460-4769-41ca-8d51-6a96e8449887     []  

 Data for  node_2034e2c7-9db4-4adc-93d5-21f07046c680 

     name data_type                tags     description               shape  \
0  MNIST   default  [#MNIST, #dataset]  MNIST database  [60000, 1, 28, 28]   

                                     dataset_id dtypes  
0  dataset_a5e182c2-2a63-4aee-aa51-7468e4c51508     []  


## Optional : clean file repository (do not run unless necessary)
Clean all the files in the repo via the rest API.

In [ ]:
# import requests
# from fedbiomed.researcher.environ import environ

# uploaded_models = requests.get(environ['UPLOADS_URL']).json()
# for m in uploaded_models:
#   requests.delete(m['url'])

Feel free to try your own models :D